In [1]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
import csv
import numpy as np
import random
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import math
from matplotlib import pyplot as plt
import collections
import Dataset as ds
import DatasetModifier as dsm

Using TensorFlow backend.


In [2]:
# Define the parameter creation steps

datasetModifier = dsm.DatasetModifier()
datasetModifier.dataset_randomize()
datasetModifier.dataset_fill_missing_number('Age', 30)
datasetModifier.dataset_categorize_number('Age', [['infant', 0, 2], ['child', 2, 10], ['teenager', 10, 18], ['youngadult', 18, 30], ['midlife', 30, 50], ['oldfart', 50, math.inf]])
datasetModifier.add_X_parameter('Age')
datasetModifier.one_hot_X_parameter('Age')
datasetModifier.add_X_parameter('Sex')
datasetModifier.one_hot_X_parameter('Sex')
datasetModifier.standardize_X()

datasetModifier.add_Y_parameter('Survived')

In [3]:
# Load the train/test dataset
dataset = ds.Dataset()
dataset.load_dataset_from_csv('train.csv')

In [5]:
dataset.find_null_columns()

hello


Age         177
Cabin       687
Embarked      2
dtype: int64

In [6]:
# Load the prediction dataset
dataset_prediction = ds.Dataset()
dataset_prediction.load_dataset_from_csv('test.csv')

In [8]:
dataset.find_null_columns()

In [9]:
# Apply the parameter creation steps to the two datasets
datasetModifier.generate_X(dataset)
datasetModifier.generate_Y(dataset)
datasetModifier.generate_X(dataset_prediction)

D:\Programs\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
D:\Programs\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a L

In [10]:
train_X, test_X = dataset.get_X_train_test_sets(0.2)
train_Y, test_Y = dataset.get_Y_train_test_sets(0.2)

In [11]:
# Fetch the predict set data
predict_X, _ = dataset_prediction.get_X_train_test_sets(0)
test_passenger_ids = dataset_prediction.get_dataset_parameter('PassengerId')
test_passenger_ids = np.reshape(test_passenger_ids.values, (test_passenger_ids.shape[0], 1))

In [26]:
results = {}

# Logistic Regression

In [27]:
from sklearn.linear_model import LogisticRegressionCV

lr = LogisticRegressionCV(cv=7, random_state=0)
lr.fit(train_X, train_Y)

lr_score = lr.score(test_X, test_Y)
print(lr_score)
results["Logistic Regression"] = {'score': lr_score,
                     'model': lr}

D:\Programs\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.7865168539325843


# Use Variations of SVM

- Apply Grid Search Cross Validation to identify best Parameters (that might take a while depending on the number of steps...)

In [28]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV

def find_svm_params():

    NUM_STEPS = 40

    param_grid = {
        'C': np.logspace(-3, 4, NUM_STEPS)
    }

    grid = GridSearchCV(svm.SVC(gamma='auto'), param_grid, cv=7)
    grid.fit(train_x, train_y)

    print(grid.best_params_)
    
#find_svm_params()

In [29]:
svc_linear = svm.SVC(kernel='linear', C=2.5719138090593443)
svc_linear.fit(train_X, train_Y)

svc_linear_score = svc_linear.score(test_X, test_Y)
print(svc_linear_score)

results["SVC Linear"] = {
    'score': svc_linear_score,
    'model': svc_linear
}

0.7865168539325843


D:\Programs\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [30]:
def find_svm_kernel_params():
    param_grid = {
        'C': np.logspace(-3, 4, NUM_STEPS),
        'gamma': np.logspace(-3, 4, NUM_STEPS),
    }

    grid = GridSearchCV(svm.SVC(kernel='rbf'), param_grid, cv=7)
    grid.fit(train_x, train_y)

    print(grid.best_params_)
    
#find_svm_kernel_params()

In [40]:
svc_rbf = svm.SVC(kernel='rbf', gamma=0.04124626382901352, C=8.886238162743407)
svc_rbf.fit(train_X, train_Y)

svc_rbf_score = svc_rbf.score(test_X, test_Y)
print(svc_rbf_score)

results["SVC RBF"] = {
    'score': svc_rbf_score,
    'model': svc_rbf
}

0.7865168539325843


D:\Programs\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


# Naive Bayes

In [32]:
from sklearn.naive_bayes import GaussianNB

naive_gauss = GaussianNB()
naive_gauss.fit(train_X, train_Y)

naive_gauss_score = naive_gauss.score(test_X, test_Y)
print(naive_gauss_score)

results["Naive Gauss"] = {
    'score': naive_gauss_score,
    'model': naive_gauss
}

0.7359550561797753


D:\Programs\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


# Classification Trees & Random Forests

In [34]:
from sklearn import tree
clf = tree.DecisionTreeClassifier(max_depth=6)

clf.fit(train_X, train_Y)
clf_score = clf.score(test_X, test_Y)
print(clf_score)

results["Classification Tree"] = {
    'score': clf_score,
    'model': clf
}

0.7696629213483146


In [35]:
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

def random_search_for_params():
    param_grid = {
        'max_features': ['auto', 'sqrt'],
        'n_estimators': [int(x) for x in np.linspace(start = 200, stop = 7000, num = 10)],
        'max_depth': [int(x) for x in np.linspace(10, 110, num = 11)],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'bootstrap': [True, False],
    }

    rf = RandomForestRegressor()
    rf_random = RandomizedSearchCV(estimator = rf, param_distributions = param_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
    rf_random.fit(train_x, train_y)

    print(rf_random.best_params_)
    
def grid_search_for_params():
    param_grid = {
        'max_features': ['auto'],
        'n_estimators': [4000, 5000, 5500, 6000, 7000],
        'max_depth': [5, 8, 10, 15, 20, 50],
        'min_samples_split': [4, 5, 6],
        'min_samples_leaf': [1],
        'bootstrap': [True],
    }

    rf = RandomForestRegressor()
    rf_gridCV = GridSearchCV(estimator = rf, param_grid = param_grid, cv = 3, verbose=2, n_jobs = -1)
    rf_gridCV.fit(train_x, train_y)

    print(rf_gridCV.best_params_)
    
#random_search_for_params()
#grid_search_for_params()

In [36]:
rfc = RandomForestClassifier(max_features='auto', max_depth=5, n_estimators=5000, min_samples_split=4, min_samples_leaf=1, bootstrap=True)
rfc.fit(train_X, train_Y)

rfc_score = rfc.score(test_X, test_Y)
print(rfc_score)

results["Random Forest"] = {
    'score': rfc_score,
    'model': rfc
}

D:\Programs\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


0.7696629213483146


In [49]:
best_score = 0

for key in results:
    model_name = key
    score = results[key]["score"]
    print("{0} scored {1}".format(model_name, score))
    
    if score > best_score:
        best_score = score
        best_model_name = model_name
        best_model = results[key]['model']
    
print('Best Model is: {0} with a score of {1} - will continue with this'.format(best_model_name, best_score))

Logistic Regression scored 0.7865168539325843
SVC Linear scored 0.7865168539325843
SVC RBF scored 0.7865168539325843
Naive Gauss scored 0.7359550561797753
Classification Tree scored 0.7696629213483146
Random Forest scored 0.7696629213483146
Best Model is: Logistic Regression with a score of 0.7865168539325843 - will continue with this


# Predict Model

In [52]:
# Run the model against the test data
predict_Y = lr.predict(predict_X)
predict_Y = np.around(predict_Y)
predict_Y = predict_Y.astype(np.integer)
predict_Y = np.reshape(predict_Y, (predict_Y.shape[0], 1))

In [53]:
csv_predict = np.concatenate((test_passenger_ids, predict_Y), axis=1)
csv_predict = np.concatenate((np.reshape(["PassengerId", "Survived"], (1, 2)), csv_predict))
with open('prediction.csv', 'w', newline='') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerows(csv_predict)
csvFile.close()